In [1]:
# Notes from reading/studying Fluent Python by Luciano Ramalho
# Chapter 10
# Created 9/22/20

In [1]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools
import string


In [4]:
a = array('d', range(10))

In [11]:
print(f'{a} is of type: {type(a)}')
b = iter(a)
print(f'{b} is of type {type(b)}')
print(f'a is b -> {a is b} \na == b -> {a==b}')
      



array('d', [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]) is of type: <class 'array.array'>
<arrayiterator object at 0x0617E448> is of type <class 'arrayiterator'>
a is b -> False 
a == b -> False


In [23]:
for i in dir(b):
    print(f'{i:20} -> {getattr(b, i)}')

__class__            -> <class 'arrayiterator'>
__delattr__          -> <method-wrapper '__delattr__' of arrayiterator object at 0x0617E448>
__dir__              -> <built-in method __dir__ of arrayiterator object at 0x0617E448>
__doc__              -> None
__eq__               -> <method-wrapper '__eq__' of arrayiterator object at 0x0617E448>
__format__           -> <built-in method __format__ of arrayiterator object at 0x0617E448>
__ge__               -> <method-wrapper '__ge__' of arrayiterator object at 0x0617E448>
__getattribute__     -> <method-wrapper '__getattribute__' of arrayiterator object at 0x0617E448>
__gt__               -> <method-wrapper '__gt__' of arrayiterator object at 0x0617E448>
__hash__             -> <method-wrapper '__hash__' of arrayiterator object at 0x0617E448>
__init__             -> <method-wrapper '__init__' of arrayiterator object at 0x0617E448>
__init_subclass__    -> <built-in method __init_subclass__ of type object at 0x7A313BC0>
__iter__            

In [24]:
for i in a:
    print(f'{i} is of type: {type(i)}')

0.0 is of type: <class 'float'>
1.0 is of type: <class 'float'>
2.0 is of type: <class 'float'>
3.0 is of type: <class 'float'>
4.0 is of type: <class 'float'>
5.0 is of type: <class 'float'>
6.0 is of type: <class 'float'>
7.0 is of type: <class 'float'>
8.0 is of type: <class 'float'>
9.0 is of type: <class 'float'>


In [70]:
from array import array
import reprlib
import math
import numbers
from functools import reduce
import operator

class Vector:
    typecode = 'd'
    shortcut_names='xyzt'
#     letters = string.ascii_lowercase

    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
               bytes(self._components))
    
    def __abs__(self):
        return math.sqrt(sum(x * x) for x in self)
    
    def __bool__(self):
        return bool(abs(self))
        
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
#         print(f'Index = {index}')
        cls = type(self)
        if isinstance(index, slice):
#             print("I'm a slice")
            x = cls(self._components[index])
#             print(x)
            return x
        elif isinstance(index, numbers.Integral):
#             print("I'm an Integral")
            return self._components[index]
        else:
            raise TypeError(f'Index must be integer or slice.\nYou selected: {index}')
    
    def __getattr__(self, attr):
        cls = type(self)
        attr = attr.lower()
        pos = cls.shortcut_names.find(attr)
        if 0 <= pos < len(cls.shortcut_names):
            return self._components[pos]
        else:
            raise AttributeError(f'Unknown attribute {attr} selected for {self!r}')
            
    def __setattr__(self, attr, value):
        cls = type(self)
        attr = attr.lower()
        pos = cls.shortcut_names.find(attr)
        if 0 <= pos < len(cls.shortcut_names):
            raise AttributeError(f'Attribute "{attr}" selected.  ' +
                                 f'"{cls.shortcut_names}" attributes are read only.')
        else:
            object.__setattr__(self, attr, value)
        
    def __hash__(self):
#         return self.hash1()
        return self.hash2()
    
    def hash1(self):
        hashes = [hash(i) for i in self._components]
#         x = reduce(lambda x, y: x^y, hashes)
        x = reduce(operator.xor, hashes, 0)
        return x

    def hash2(self):
        hashes = map(hash, self._components)
        x = reduce(operator.xor, hashes, 0)
        return x

    def __eq__(self, other):
        if len(self._components) != len(other):
            return False
        for i, j in zip(self._components, other):
            if i != j:
                return False
        return True


In [71]:
a = Vector([1, 2, 3, 4, 5, 6])
print(hash(a))

7


In [73]:
b = Vector([1, 2])
c = Vector([1, 2])
d = Vector([1, 3])
print(b==c, b==d)

True False


In [50]:
a = Vector([3.1, 4.2, 3, 4, 5, 6])
print(a.x)
print(a.y)
print(a.z)
print(a.t)
# print(a.p)

3.1
4.2
3.0
4.0


In [51]:
a.p = 10

In [52]:
vars(a)

{'_components': array('d', [3.1, 4.2, 3.0, 4.0, 5.0, 6.0]), 'p': 10}

In [53]:
a.x = 15

AttributeError: Attribute "x" selected.  "xyzt" attributes are read only.

In [54]:
print(a[2])
a[2]=100
vars(a)

3.0


TypeError: 'Vector' object does not support item assignment

In [43]:
a = Vector([3.1, 4.2])
print("str  -> {0!s}\nrepr -> {0!r}".format(a))

str  -> (3.1, 4.2)
repr -> Vector([3.1, 4.2])


In [44]:
b = a._components

In [45]:
components = reprlib.repr(b)
print(components)
c = components.find('[')
print(c)
d = components[components.find('['):-1]
print(d)


array('d', [3.1, 4.2])
11
[3.1, 4.2]


In [48]:
v1 = Vector([3, 4, 5])
print(len(v1))
print(v1[0], v1[-1])
v7 = Vector(range(7))
print(v7[1:4])

3
3.0 5.0
array('d', [1.0, 2.0, 3.0])


In [49]:
# Notes on how slices work
class MySeq():
    def __getitem__(self, index):
        return index

s = MySeq()
print(s[1])
print(s[1:4])
print(s[1:4:2])
print(s[1:4:2, 9])
print(s[1:4:2, 7:9])


1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


In [55]:
print(v7[1:4])

array('d', [1.0, 2.0, 3.0])


In [71]:
v8 = Vector(range(8))
print("{0!r}\n{0!s}".format(v8[1:4]))

Index = slice(1, 4, None)
I'm a slice
(1.0, 2.0, 3.0)
Vector([1.0, 2.0, 3.0])
(1.0, 2.0, 3.0)


In [72]:
v8[1,2]

Index = (1, 2)


TypeError: Index must be integer or slice./nYou selected: (1, 2)